In [1]:
import requests
import os
import pandas as pd
from datetime import date, datetime, timedelta
import time
from io import StringIO
idag = date.today().isoformat()[0:10]
tokenCanvas = os.environ['tokenCanvas']

headers = {'Authorization': f'Bearer {tokenCanvas}',}
params = {'per_page': 100,}

# Liste alle emne

Først lister eg alle emne, og så finn eg alle som har "2025_HØST" i seg. Det dekker i alle fall dei som starter opp no i haust. Dei som er andre semester av eit to-semestersemne er ikkje så viktige, for dei er vel mest sannsynleg publisert allereie?

In [17]:
url = f"https://hvl.instructure.com/api/v1/accounts/1/courses"

def get_courses():
    courses = []
    page = 1
    while page < 3:
        params['page'] = page
        response = requests.get(url, headers=headers, params=params)
        if response.status_code != 200:
            print(f"Error fetching courses: {response.status_code}")
            break
        data = response.json()
        if not data:
            break
        courses.extend(data)
        page += 1
    return courses

emne = get_courses()

# Filtrer emne som starter i 2025_HØST
emne_2025_host = [c for c in emne if 'HØST_2017' in c['sis_course_id']]

TypeError: argument of type 'NoneType' is not iterable

In [9]:
emne1 = 34024
emne2 = 34025
url1 = f"https://hvl.instructure.com/api/v1/courses/{emne1}/settings"
url2 = f"https://hvl.instructure.com/api/v1/courses/{emne2}/settings"
def get_course_settings(course_id):
    url = f"https://hvl.instructure.com/api/v1/courses/{course_id}/settings"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching course settings for {course_id}: {response.status_code}")
        return None
    
settings1 = get_course_settings(emne1)
settings2 = get_course_settings(emne2)

In [10]:
settings1

{'allow_final_grade_override': False,
 'allow_student_discussion_topics': True,
 'allow_student_forum_attachments': True,
 'allow_student_discussion_editing': True,
 'allow_student_discussion_reporting': True,
 'allow_student_anonymous_discussion_topics': False,
 'filter_speed_grader_by_student_group': False,
 'grading_standard_enabled': False,
 'grading_standard_id': None,
 'grade_passback_setting': None,
 'allow_student_organized_groups': True,
 'hide_final_grades': False,
 'hide_distribution_graphs': False,
 'hide_sections_on_course_users_page': False,
 'lock_all_announcements': False,
 'usage_rights_required': False,
 'restrict_student_past_view': False,
 'restrict_student_future_view': False,
 'restrict_quantitative_data': False,
 'show_announcements_on_home_page': True,
 'home_page_announcement_limit': 3,
 'syllabus_course_summary': False,
 'homeroom_course': False,
 'image_url': None,
 'image_id': None,
 'image': None,
 'banner_image_url': None,
 'banner_image_id': None,
 'banne

In [11]:
url11 = f"https://hvl.instructure.com/api/v1/courses/{emne1}"
url21 = f"https://hvl.instructure.com/api/v1/courses/{emne2}"

def get_course_details(course_id):
    url = f"https://hvl.instructure.com/api/v1/courses/{course_id}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching course details for {course_id}: {response.status_code}")
        return None

course_details1 = get_course_details(emne1)
course_details2 = get_course_details(emne2)

In [15]:
course_details2

{'id': 34025,
 'name': 'MAT122 Førde - Informasjonsteknologi (11)',
 'account_id': 18953,
 'uuid': 'Mp6gylYGqA42SVObiCD1RNhrXdh4OfdFDdi9hopN',
 'start_at': None,
 'grading_standard_id': None,
 'is_public': False,
 'created_at': '2025-06-04T10:11:33Z',
 'course_code': 'MAT122 Førde - Informasjonsteknologi (11)',
 'default_view': 'syllabus',
 'root_account_id': 1,
 'enrollment_term_id': 353,
 'license': 'private',
 'grade_passback_setting': None,
 'end_at': None,
 'public_syllabus': False,
 'public_syllabus_to_auth': False,
 'storage_quota_mb': 524,
 'is_public_to_auth_users': False,
 'homeroom_course': False,
 'course_color': None,
 'friendly_name': None,
 'hide_final_grades': False,
 'apply_assignment_group_weights': False,
 'calendar': {'ics': 'https://hvl.instructure.com/feeds/calendars/course_Mp6gylYGqA42SVObiCD1RNhrXdh4OfdFDdi9hopN.ics'},
 'time_zone': 'Europe/Copenhagen',
 'blueprint': False,
 'template': False,
 'sis_course_id': 'UA_203_MAT122_1_2025_HØST_1_11',
 'sis_import_id':